# Чтение и обработка словарей из pdf

Использован модуль PyPDFLoader для чтения pdf, затем обработан текст на выделение слова с ударением и по возможности его одной словоформы в отдельный словарь.

## Обработка словаря Новый орфоэпический словарь русского языка для всех, кто хочет быть грамотным/ Т.А. Гридина, Н.И. Коновалова, В.В. Бурцева. — Москва: АСТ, 2013

In [1]:
import re
from langchain.document_loaders import PyPDFLoader

In [3]:
loader = PyPDFLoader("Gridina_T._Noviy_orfoepicheskiy_slovar_russkogo_yazika.pdf")
pages = loader.load_and_split()

In [4]:
# первый раунд чистки
words = []
for page in pages[13:638]:
    content = page.page_content.replace(' /accent', '+').replace('фaccent', '+').replace('/accent', '+')
    content = content.replace(' /box2', '').replace('/box2', '')
    content = re.sub("[\(\[].*?[\)\]]", "", content)
    lines = content.split('\n')
    for line in lines:
        if ';' in line:
            word = line.split(';')[0]
            
            word = re.sub('[0-9]+', '', word)
            words.append( ''.join(word.split()))

In [5]:
#второй раунд чистки
bad_items = ['.', '...', ']', ':', ')']
new = []
for word in words:
    splt = word.split(',')
    for i in splt:
        index = i.count('+')
        if index == 1:
            if not any(x in i for x in bad_items):
                new.append(i)
new[0:5]

['абажу+р', 'абажу+ра', 'аббревиату+ра', 'аббревиату+ры', 'абза+ц']

In [6]:
#удалим слова, которые не по алфавиту и ошибочны

fin_list = []
for i in new:
    index = new.index(i)
    
    if (index !=0) and (index != len(new) - 1):
        previous = index - 1
        next_element = index +1
        if (new[index][0] == new[previous][0]) or (new[index][0] == new[next_element][0]):
            accent_index = i.find('+')
            i = i[:accent_index-1]+ '+' + i[accent_index-1] + i[accent_index+1:]
            fin_list.append(i)

    else:
        accent_index = i.find('+')
        i = i[:accent_index-1]+ '+' + i[accent_index-1] + i[accent_index+1:]
        fin_list.append(i)
fin_list[:5]

['абаж+ур', 'абаж+ура', 'аббревиат+ура', 'аббревиат+уры', 'абз+ац']

In [7]:
import pandas as pd
df = pd.DataFrame(fin_list, columns = ['with_accent'])
df['raw_text'] = df['with_accent'].apply(lambda x: re.sub('[^а-яёА-ЯЁ]+', '', x))
df = df[['raw_text', 'with_accent']]
df[0:5]

,raw_text,with_accent
0,абажур,абаж+ур
1,абажура,абаж+ура
2,аббревиатура,аббревиат+ура
3,аббревиатуры,аббревиат+уры
4,абзац,абз+ац


In [8]:
df.to_csv('dict1.csv')

In [9]:
import spacy
ru_nlp = spacy.load('ru_core_news_md')

In [10]:
raw_lst = list(df['raw_text'])
lemmas_dict = {}
for item in raw_lst:
    lemma = ru_nlp(item)[0].lemma_
    lemmas_dict[lemma] = lemmas_dict.get(lemma, 0) + 1
    
len(lemmas_dict)

3088

Итого было выгружено 4 746 слов, с уникальной леммой - 3088

## Обработка Орфоэпический словарь русского языка: правильно ли мы говорим? / О. А. Михайлова. — Москва : Издательство АСТ, 2022

In [11]:
loader2 = PyPDFLoader("Mihaylova_O._Orfoepicheskiy_slovar_russkogo_yazika.pdf")
pages2 = loader2.load_and_split()

In [12]:
words = []
for page in pages2[10:157]:
    content = page.page_content.split('\n')
    for line in content:
        if ',' in line:
            words.append(line.split(',')[0].strip())
words[:5]

['агресси́вный', 'агре́ссия', 'агроно́мия', 'ада́птер', 'адеква́тный']

In [13]:
#второй раунд чистки
bad_items = ['.', '...', ']', ':', ')', '(', '`']
new = []
for word in words:
     if not any(x in word for x in bad_items):
            if word != '':
                new.append(word.strip())
new[:5]

['агресси́вный', 'агре́ссия', 'агроно́мия', 'ада́птер', 'адеква́тный']

In [14]:
#удалим слова, которые не по алфавиту и ошибочны

fin_list = []
for i in new:
    index = new.index(i)
    
    if (index !=0) and (index != len(new) - 1):
        previous = index - 1
        next_element = index +1
        if (new[index][0] == new[previous][0]) or (new[index][0] == new[next_element][0]):
            #accent_index = i.find('+')
            #i = i[:accent_index-1]+ '+' + i[accent_index-1] + i[accent_index+1:]
            fin_list.append(i)

    else:
        #accent_index = i.find('+')
        #i = i[:accent_index-1]+ '+' + i[accent_index-1] + i[accent_index+1:]
        fin_list.append(i)
fin_list[:5]

['агресси́вный', 'агре́ссия', 'агроно́мия', 'ада́птер', 'адеква́тный']

In [15]:
# выделим в отдельный список те слова, которые имеют несколько вариантов ударений
multiple_accents = []
for i in fin_list:
    if ' и ' in i:
        lst = i.split(' и ')
        for item in lst:
            item = re.sub('[^а-яёА-ЯЁ+ ]+', '', item)
            multiple_accents.append(f'{item}: {lst}')
multiple_accents = list(dict.fromkeys(multiple_accents))
multiple_accents[:5]

["ала: ['ала́', 'а́ла']",
 "баржам: ['баржа́м', 'ба́ржа']",
 "баржа: ['баржа́м', 'ба́ржа']",
 "батуд: ['бату́д', 'бату́т']",
 "батут: ['бату́д', 'бату́т']"]

In [16]:
# удалим слова, где есть пробелы
fin = []
for i in fin_list:
    if ' ' not in i:
        fin.append(i)
len(fin)       

5236

In [17]:
#поставим вместо ударения знак +
accented = ['а́', 'я́', 'у́', 'ю́', 'о́', 'е́', 'э́', 'и́', 'ы́', 'ё']

fin_last = []

for word in fin:
    accent_index = [word.find(x) for x in accented if word.find(x) >= 0]
    if len(accent_index) == 0:
        fin_last.append(word)
    elif len(accent_index) == 1:
        word_new = word[:accent_index[0]]+'+'+word[accent_index[0]:]
        word_new = re.sub('[^а-яёА-ЯЁ+]+', '', word_new)
        fin_last.append(word_new)
    else:
        word_new = word[:accent_index[0]]+'+'+word[accent_index[0]:]
        for i in range(1, len(accent_index)):
            #word = test[:accent_index[0]]+'+'+test[accent_index[0]:]
            word_new = word_new[:accent_index[i]+i]+'+'+word_new[accent_index[i]+i:]
            word_new = re.sub('[^а-яёА-ЯЁ+]+', '', word_new)
            fin_last.append(word_new)
fin_last[0:5]

['агресс+ивный', 'агр+ессия', 'агрон+омия', 'ад+аптер', 'адекв+атный']

In [18]:
df = pd.DataFrame(fin_last, columns = ['with_accent'])
df['raw_text'] = df['with_accent'].apply(lambda x: re.sub('[^а-яёА-ЯЁ]+', '', x))
df = df[['raw_text', 'with_accent']]
df[0:5]

,raw_text,with_accent
0,агрессивный,агресс+ивный
1,агрессия,агр+ессия
2,агрономия,агрон+омия
3,адаптер,ад+аптер
4,адекватный,адекв+атный


In [19]:
df.to_csv('dict2.csv')

In [20]:
raw_lst = list(df['raw_text'])
lemmas_dict = {}
for item in raw_lst:
    lemma = ru_nlp(item)[0].lemma_
    lemmas_dict[lemma] = lemmas_dict.get(lemma, 0) + 1
    
len(lemmas_dict)

4539

In [21]:
mult_acc = []

for word in multiple_accents:
    accent_index = [[m.start() for m in re.finditer(i, word)] for i in accented if len([m.start() for m in re.finditer(i, word)]) >= 0 ]
    accent_index = [item for row in accent_index for item in row]
    
    if len(accent_index) == 0:
        mult_acc.append(word)
    elif len(accent_index) == 1:
        word_new = word[:accent_index[0]]+'+'+word[accent_index[0]:]
        word_new = re.sub('[^а-яёА-ЯЁ+:, ]+', '', word_new)
        mult_acc.append(word_new)
    else:
        word_new = word[:accent_index[0]]+'+'+word[accent_index[0]:]
        for i in range(1, len(accent_index)):
            #word = test[:accent_index[0]]+'+'+test[accent_index[0]:]
            word_new = word_new[:accent_index[i]+1]+'+'+word_new[accent_index[i]+1:]
            word_new = re.sub('[^а-яёА-ЯЁ+:, ]+', '', word_new)
            mult_acc.append(word_new)
    
mult_acc[:5]

['ала: ал+а, +ала',
 'баржам: барж+ам, б+аржа',
 'баржа: барж+ам, б+аржа',
 'батуд: бат+уд, бат+ут',
 'батут: бат+уд, бат+ут']

In [ ]:
df = pd.DataFrame(mult_acc, columns = ['mult_accents'])
df.to_csv('dict2_multiple_accents.csv')